# InfCtx trainer validation
This model being trained has the same settings as raven 1B5 model.
- Layer count: 24
- Embed size: 2048

The goal is to validate loss rate change, across the exact same hyper parameters
- 1024 data chunk size
- same learningrate / weightdecay / seed

With only the change in working context size
- 1024 context vs 128 context

> This project assumes you have the rwkv-exp conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

## Prepare the dataset

In [1]:
# First lets get the blank init model, these init model was generated
# using the original RWKV-LM repo (as at this point of writing, this repo cannot init a model)
#
# As such I have preinitialized these blank models and uploaded them to HF for convinence
!mkdir -p ./model/
!rm -rf ./model/Echo-A-1B5-Init.pth
!cd ./model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Init.pth
!ls -alh ./model/Echo-A-1B5-Init.pth

--2023-06-26 01:59:02--  https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Init.pth
Resolving huggingface.co (huggingface.co)... 18.154.227.87, 18.154.227.69, 18.154.227.67, ...
Connecting to huggingface.co (huggingface.co)|18.154.227.87|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/cb/ef/cbef09abb2634a3375b28868bffa285226dfeabedec89b28c2fb302221164d66/0ec7214ed16737a6348254e6f96d8cdc04d3b5efbd5f53fe9337607ea42b5b9f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Echo-A-1B5-Init.pth%3B+filename%3D%22Echo-A-1B5-Init.pth%22%3B&Expires=1688003942&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2NiL2VmL2NiZWYwOWFiYjI2MzRhMzM3NWIyODg2OGJmZmEyODUyMjZkZmVhYmVkZWM4OWIyOGMyZmIzMDIyMjExNjRkNjYvMGVjNzIxNGVkMTY3MzdhNjM0ODI1NGU2Zjk2ZDhjZGMwNGQzYjVlZmJkNWY1M2ZlOTMzNzYwN2VhNDJiNWI5Zj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb2

In [2]:
# Lets ensure some of the directories we need are setup
!mkdir -p ./datapaths/
!mkdir -p ./checkpoint/

In [3]:
# Lets preload the requried dataset
!cd ./RWKV-v4neo && python3 preload_dataset.py ./Mini-Enwiki-Test-A.yaml

Extracting data files: 100%|████████████████████| 1/1 [00:00<00:00, 2162.01it/s]
Setting num_proc from 32 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Dataset parquet downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 1032.32it/s]


## Training Context Size 1024

In [5]:
# Lets start the training process
!cd ./RWKV-v4neo && python new_train.py fit -c ./Mini-Enwiki-Test-A.yaml

Setting ds_accelerator to cuda (auto detect)
Global seed set to 3941088705
usage: new_train.py [-h] [-c CONFIG] [--print_config[=flags]]
                    {fit,validate,test,predict} ...
new_train.py: error: 'Configuration check failed :: No action for destination key "dataset_loader.test" to check its value.'


## Training Context Size 128

In [4]:
# Lets start the training process
!cd ./RWKV-v4neo && python new_train.py fit -c ./Mini-Enwiki-Test-B.yaml

Setting ds_accelerator to cuda (auto detect)
Global seed set to 3941088705
Global seed set to 3941088705
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230626_121748-aenwikm32
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Mini-Enwiki-Test-B (no deepspeed checkpoint, lr 3e-4/1e-5/5, wd 0.01, ctx 128, data 1024, fixed seed, bs 16)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Memory-Experiment
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Memory-Experiment/runs/aenwikm32
Using /home/ubuntu/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu117/wkv_128_bf16/build.ninja...
Building extension module wkv_128_bf16...
Allowing ninja to set a default number of workers... (overridable by setting